In [148]:
import numpy as np
import pandas as pd

In [149]:
train = pd.read_csv("./train.csv")
features = train.drop(columns="quality")
print(features.keys())
keys = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides",'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

Index(['type', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [150]:
dim = len(keys)

In [151]:
X_label = np.ndarray(shape=(1 + dim,len(features)),dtype=float)

for i in range(len(features)):
    X_label[0][i] = 1.0
for d, key in enumerate(keys):
    X_label[d+1] = [x if x != "red" and x != "white" else (1.0 if x == "red" else 0.0) for x in features[key].values]

Y_label = train["quality"].values
Y_label = Y_label.reshape((1,len(Y_label)))
# print(X_label.shape,X_label)
# print(Y_label.shape,Y_label)

In [152]:
B = np.where(np.isnan(X_label), 0, X_label).dot(np.where(np.isnan(Y_label.T), 0, Y_label.T))
A = np.where(np.isnan(X_label), 0, X_label).dot(np.where(np.isnan(X_label.T), 0, X_label.T))

L = 0.8 # 正則化の程度を決めるハイパーパラメータ
LI = np.identity(1 + dim) * L

AT = np.linalg.inv(A + LI)
w = np.where(np.isnan(AT), 0, AT).dot(np.where(np.isnan(B), 0, B))
# print(B)
# print(A)
# print(AT)
print(w)

[[ 1.16994897]
 [ 0.01068944]
 [-1.44663831]
 [-0.15842369]
 [ 0.02242785]
 [-0.71410421]
 [ 0.00597784]
 [-0.00221716]
 [ 0.99999948]
 [ 0.08256702]
 [ 0.63779143]
 [ 0.33381725]]


In [153]:
# output prediction for the test data
test = pd.read_csv("./test.csv")

In [154]:
submission = pd.read_csv("./sample_submission.csv", index_col="id")
submission["quality"] = [w[0][0] + sum(w[i + 1][0] * (test[key][n] if key != "type" else (1.0 if test[key][n] == "red" else 0.0)) for i, key in enumerate(keys)) for n in range(len(test))]
submission.to_csv("submission.csv")